<a href="https://colab.research.google.com/github/joaoBernardinoo/CellAdapt/blob/main/atividade_01_formas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Níveis de Analise Linguistica e Análise Lexical e Morfológica Automatizada - Tokenização, Stemming e Lematização**

## Tokenização, Stemming (derivação) e Lematização

### Links úteis

[NLTK tutorial](https://data-flair.training/blogs/nltk-python-tutorial/)

[Python stemming and lemmatization](https://data-flair.training/blogs/python-stemming/)

[NLTK for portuguese](https://www.nltk.org/howto/portuguese_en.html)

[Portuguese lemmatizers](https://lars76.github.io/2018/05/08/portuguese-lemmatizers.html)

## Sumário
- Tokenização
- Lematização
- Stemming
- Níveis de análise linguística
  - Fonética
  - Morfológica
  - Sintática
  - Semântica
  - Pragmática
  - Texto de exemplo
    - Árvore de dependência

## Níveis de Análise Linguística

<img src="https://images.slideplayer.com.br/40/11157777/slides/slide_39.jpg" width="80%">

### Morfológica

É o estudo da estrutura das palavras e suas classificações em diferentes categorias.

Considere os exemplos:

- árvore<br>
- árvore<ins>s</ins><br>
- arvore<ins>zinhas</ins><br>
- <ins>im</ins>possível<br>
- <ins>sobre</ins>mesa

Essas palavras são formadas por morfemas, que se dividem em independentes e dependentes.

 - *árvore*, *possível*, *sobre* e *mesa* são **morfemas independentes**.

 - *s*, *zinhas* e *im* são **morfemas dependentes**.


As palavras podem ser classificadas em partes do discurso (part-of-speech -> POS).

***substantivos, verbos, adjetivos, preposições e advérbios*** são exemplos dessas partes do discurso.

Essas classes também podem ser agrupadas entre:
 - abertas, que abrangem um grande número de palavras e abrigam facilmente novas entradas, como substantivos e verbos;
 - fechadas, que possuem funções gramaticais bem definidas, como artigos e preposições.



### Sintática

É o estudo do sentido entre as palavras bem como a sua disposição em uma frase.

Com a classificação morfológica das palavras, uma análise sintática demonstra a validade do texto de acordo com a gramática empregada.

A análise sintática também permite reconhecer os termos da oração:

- Termos essenciais
 - Sujeito
 - Predicado

- Termos integrantes
 - Complemento verbal
 - Complemento nominal
 - Agente da passiva

- Acessórios
 - Adjunto adnominal
 - Adjunto adverbial
 - Aposto





<img src="https://linguisticageralunip.files.wordpress.com/2017/11/sintaxe-volume-1-2-638.jpg" width="40%">





### Exemplo prático
Abaixo podemos ver na prática a execução das análises morfológicas e sintáticas e suas diferenças e relações.

In [ ]:
import pickle
import nltk
#download do punkt e rslp (2 pacotes que serão utilizados)

In [ ]:
import spacy

import spacy.cli
spacy.cli.download("pt_core_news_sm")

nlp = spacy.load("pt_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Fazendo nosso próprio analisador morfológico

In [ ]:
!pip install conllu
!wget http://marlovss.work.gd:8080/tomorrow/aula2/bosque.conllu

--2024-10-22 17:43:09--  http://marlovss.work.gd:8080/tomorrow/aula2/bosque.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.148.12
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.148.12|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11291250 (11M)
Saving to: ‘bosque.conllu.1’

bosque.conllu.1     100%[===================>]  10.77M  1.85MB/s    in 11s     

2024-10-22 17:43:20 (1.02 MB/s) - ‘bosque.conllu.1’ saved [11291250/11291250]



In [ ]:
import conllu
import itertools as it

class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


class CoNLLU:
   def __init__(self, files):
      self.words = []
      self.sentences = []
      for f in files:
         parsed = conllu.parse(open(f).read())
         sents = [[AttributeDict(form = token['form'], lemma=token['lemma'],pos=token['upos'],feats=token['feats']) for token in tokenlist if token['upos']!='_'] for tokenlist in parsed]
         self.sentences.extend(sents)
         self.words.extend([word for sent in sents for word in sent])
      self.pos_tags = set([word.pos for word in self.words])
      self.feats_dict ={pos:set(it.chain.from_iterable([list(word.feats.keys()) for word in self.words if word.pos==pos and word.feats!= None])) for pos in self.pos_tags}


In [ ]:
bosque = CoNLLU(files=["bosque.conllu"])

## Vamos começar com um etiquetador morfossintático (POS tagger).

Um etiquetador morfossintático tem por objetivo identificar a classe gramatical das palavras em uma sentença, ou conjunto de sentenças. Por exemplo, considere a frase:

O rato roeu a roupa do rei de Roma.

Após tokenizada, temos:
O, rato, roeu, a, roupa, de, o, rei, de, Roma

O resultado do etiquetador sobre esse conjunto é:
(O, DET), (rato, NOUN), (roeu, VERB), (a, DET), (roupa, NOUN), (de, ADP), (o,DET), (rei, NOUN), (de, ADP), (Roma, PROPN)

O conjunto de etiquetas (tagset) que descrevem as classes gramaticais é determinado no projeto do corpus e **não constituem um conjunto universal ou objetivo**, mas uma decisão teórico-metodológica do projeto.

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from nltk.probability import FreqDist
suffixes = set([word.form.lower()[-3:] for word in bosque.words])


In [ ]:
#suf_to_tag = {suf:FreqDist([word.pos for word in bosque.words if word.form.lower()[-3:]==suf]).max() for suf in suffixes}

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/suf_to_tag.pkl', 'rb') as f:
    suf_to_tag = pickle.load(f)

In [ ]:
print(suffixes)

{'ndi', ',47', 'nan', "l'", 'gen', 'vro', 'edã', 'ijo', 'jk', 'nó', 'fez', 'iya', 'fat', 'uki', 'ems', 'ali', 'dro', 'ptb', 'rif', "k's", 'lvê', '974', 'lta', 'prd', 'ibi', 'jor', 'ião', 'éus', 'nga', 'stf', 'xon', 'ens', ',03', 'rvs', 'son', 'uno', 'ubo', 'azê', '3,0', '929', 'kih', 'tsu', 'cae', '9', 'aat', 'fka', 'asy', 'sci', 'pes', 'nny', 'bwv', 'h43', 'leu', 'igi', 'pie', ',54', 'poe', "66'", 'com', 'onk', '0,8', 'ntz', 'ven', 'pus', 'ove', 'pin', 'te', '?', 'jol', '25', "38'", 'pp', '120', 'tv2', '6,7', 'aio', 'nns', 'pio', 'azz', '2-1', 'fil', '46s', 'etz', 'êra', 'ele', 'ruy', 'úva', '887', 'fc', 'jet', 'gro', 'mês', 'hon', 'hev', 'îne', 'pol', 'sye', 'odo', '94', 'aña', 'zil', 'vor', 'axi', 'sós', 'uxa', 'gis', 'uce', 'afá', 'ram', '2', 'prc', 'mpa', '/60', 'côa', 'nc.', ',', 'mit', '210', 'key', '8º', 'iaf', 'cat', 'úde', '21', 'rry', 'mic', ',52', 'kg', 'caz', 'tts', 'uém', ',96', 'trô', '783', 'ceu', 'úri', 'gui', 'riu', 'ety', '92', 'bri', 'mol', 'mis', 'rez', 'rot', 'ncy

In [ ]:
print(suf_to_tag)

{'m.': 'PROPN', 'uía': 'VERB', 'l': 'PROPN', 'ays': 'PROPN', '909': 'NUM', '3-3': 'NUM', '42': 'NUM', 'la': 'PRON', 'fta': 'NOUN', 'ald': 'PROPN', 'cht': 'PROPN', 'cr$': 'SYM', '889': 'NUM', '602': 'NUM', 'vas': 'ADJ', 'man': 'PROPN', 'eña': 'PROPN', 'trá': 'VERB', 'owe': 'PROPN', 'ael': 'PROPN', '051': 'NUM', '69': 'NUM', 'jet': 'X', 'até': 'ADP', 'see': 'PROPN', 'née': 'NOUN', 'onx': 'PROPN', '6.2': 'NUM', 'hew': 'PROPN', '12': 'NUM', '810': 'PROPN', 'rêa': 'PROPN', 'uja': 'DET', 'fly': 'PROPN', 'zão': 'NOUN', 'oki': 'PROPN', 'voz': 'NOUN', 'sê': 'AUX', 'taz': 'NOUN', '7,9': 'NUM', 'ott': 'PROPN', ',92': 'NUM', '30': 'NUM', 'tre': 'ADP', 'pam': 'VERB', 'mpe': 'VERB', 'and': 'PROPN', ',95': 'NUM', 'c': 'NOUN', 'bou': 'VERB', '7,7': 'NUM', 'hvi': 'PROPN', 'ixa': 'NOUN', 'dan': 'PROPN', '4º.': 'ADJ', 'lto': 'NOUN', '4x2': 'NOUN', 'izá': 'VERB', 'spe': 'PROPN', 'st.': 'PROPN', 'ffi': 'PROPN', 'pei': 'VERB', 'ets': 'PROPN', '10h': 'NOUN', '104': 'NUM', 'unn': 'PROPN', 'bie': 'PROPN', 'rum

In [278]:
import pandas as pd

# Dicionário de regras
rules = {
    'ADJ': ["grande", "pequeno", "sofredor", "excelente"],
    'ADP': ["de", "em", "para", "com", "sobre", "sob"],
    'ADV': ["não", "muito", "aqui", "lá", "hoje", "sempre"],
    'AUX': [],
    'CCONJ': ["e", "ou", "mas", "porque", "portanto"],
    'DET': ["o", "a", "os", "as", "um", "uma"],
    'INTJ': ["oh", "ah", "olá", "tchau"],
    'NOUN': ["carro", "casa", "livro", "pessoa", "df"],
    'NUM': ["um", "dois", "três", "dez", "cem"],
    'PART': ["não"],
    'PRON': ["eu", "tu", "ele", "ela", "nós", "eles"],
    'PROPN': ["Maria", "João", "Brasil", "Lisboa"],
    'PUNCT': [".", ",", "!", "?", ";", ":", "--", "«", "»"],
    'SCONJ': ["que", "se", "como", "quando", "porque"],
    'SYM': ["+", "-", "*", "/", "%", "="],
    'VERB': [],
    'X': []
}

df = pd.DataFrame(list(rules.items()), columns=['pos_tag', 'token'])
df['token'] = df['token'].apply(set)


In [279]:
def tag(tokens):
  tagged = []
  for token in tokens:
    if token.lower() in rules['ADP']:
       tagged.append((token,"ADP"))
    elif token.lower() in rules['PRON']:
       tagged.append((token,"PRON"))
    elif token.lower() in rules['NOUN']:
       tagged.append((token,"NOUN"))
    elif token.lower() in rules['VERB']:
       tagged.append((token,"VERB"))
    elif token.lower() in rules['ADJ']:
       tagged.append((token,"ADJ"))
    elif token.lower() in rules['DET']:
       tagged.append((token,"DET"))
    elif token.lower() in rules['PUNCT']:
       tagged.append((token,"PUNCT"))
    elif token.lower() in rules['PROPN']:
        tagged.append((token,"PROPN"))
    # elif token.lower()[-3:] in suffixes:
    #   tagged.append((token,suf_to_tag[token.lower()[-3:]]))
    else:
       tagged.append((token,"_"))
  return tagged

In [ ]:
!wget http://marlovss.work.gd:8080/tomorrow/aula2/test.conllu

--2024-10-22 17:43:41--  http://marlovss.work.gd:8080/tomorrow/aula2/test.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.148.12
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.148.12|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819980 (1.7M)
Saving to: ‘test.conllu’

test.conllu         100%[===================>]   1.74M   475KB/s    in 3.7s    

2024-10-22 17:43:45 (475 KB/s) - ‘test.conllu’ saved [1819980/1819980]



In [280]:
test = CoNLLU(files=["test.conllu"])
test_sents = [[word.form for word in sent] for sent in test.sentences]
gold = [[(word.form.lower(),word.pos) for word in sent] for sent in test.sentences]
predicted = [tag(sent) for sent in test_sents]

In [281]:
def accuracy(predicted,gold):
   acertos = len([predicted[i][j][1] for i in range(len(gold)) for j in range(len(gold[i])) if predicted[i][j][1]==gold[i][j][1]])
   totais = sum([len(sent) for sent in gold])
   return acertos/totais

def abrangencia(predicted,gold):
  tagged_tokens = 0
  for sent in predicted:
    for _, predicted_tag in sent:
      if predicted_tag != "_":
        tagged_tokens += 1
  total_tokens = 0
  for sent in gold:
    for _, gold_tag in sent:
      if gold_tag != "_":
        total_tokens += 1

  return tagged_tokens / total_tokens

def F(predicted,gold):
  return 2 * (abrangencia(predicted,gold) * accuracy(predicted,gold)) / (abrangencia(predicted,gold) + accuracy(predicted,gold))

In [145]:
def validate():
  gold = [[(word.form.lower(),word.pos) for word in sent] for sent in test.sentences]
  predicted = [tag(sent) for sent in test_sents]
  return {
        'accuracy': accuracy(predicted,gold),
        'coverage': abrangencia(predicted,gold),
        "F" : F(predicted,gold)
}

In [ ]:

print("Etiquetador utlizando tags do bosque + regras arbitrárias")
validate()

Etiquetador utlizando tags do bosque + regras arbitrárias


{'accuracy': 0.778945080423127,
 'coverage': 0.9905448485726707,
 'F': 0.8720931654830408}

In [146]:
print("Etiquetador com regras arbitrárias")
validate()

Etiquetador com regras arbitrárias


{'accuracy': 0.3767569917403275,
 'coverage': 0.40802057672801045,
 'F': 0.39176605252931807}

In [282]:
def humanTaggingHelper():
    tagged_sent = next((sent for sent in predicted if any(tag == "_" for _, tag in sent)), None)
    if tagged_sent:
        print(tagged_sent)
        tokens = [token for token, _ in tagged_sent]
        doc = spacy.tokens.Doc(nlp.vocab, words=tokens)
        for token, (text, tag) in zip(doc, tagged_sent):
            if tag != "_":
                token.pos_ = tag
            else:
                token.tag_ = tag
        displacy.render(doc, style='dep', jupyter=True, options={'distance': 70})

In [288]:
def add_rule_to_df(df, category, token):
    token_lower = token.lower()
    idx = df[df['pos_tag'] == category].index[0]
    df.iloc[idx,1].insert(token_lower)
    print(f"'{token_lower}' adicionado à regra {category}")

In [283]:
categories = df.iloc[:,0]

def humanTaggingHelper():
    tagged_sent = next((sent for sent in predicted if any(tag == "_" for _, tag in sent)), None)
    if tagged_sent:
        print(tagged_sent)
        tokens = [token for token, _ in tagged_sent]
        doc = spacy.tokens.Doc(nlp.vocab, words=tokens)
        for token, (text, tag) in zip(doc, tagged_sent):
            if tag != "_":
                token.pos_ = tag
            else:
                token.tag_ = tag
        displacy.render(doc, style='dep', jupyter=True, options={'distance': 70})

        print("Mapeamento de Categorias:")
        for i, category in enumerate(categories):
            print(f"{i} - {category}")

        for token, (text, tag) in zip(doc, tagged_sent):
            if tag == "_":
                new_tag = input(f"Qual é a categoria de '{text}'? (Q para sair) ").strip().upper()
                if new_tag == "Q":
                    print("Encerrando o programa.")
                    return
                if new_tag.isdigit() and int(new_tag) < len(categories):
                    category = categories[int(new_tag)]
                    add_rule_to_df(df, category, text)
                else:
                    print(f"Entrada inválida ou categoria não encontrada.")

In [289]:
humanTaggingHelper()

[('Folha', '_'), ('--', 'PUNCT'), ('Como', '_'), ('você', '_'), ('recebeu', '_'), ('a', 'DET'), ('notícia', '_'), ('de', 'ADP'), ('que', '_'), ('seria', '_'), ('substituído', '_'), ('?', 'PUNCT')]


/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:141: UserWarning: [W005] Doc object not parsed. This means displaCy won't be able to generate a dependency visualization for it. Make sure the Doc was processed with a model that supports dependency parsing, and not just a language class like `English()`. For more info, see the docs:
https://spacy.io/usage/models
  warnings.warn(Warnings.W005)


Mapeamento de Categorias:
0 - ADJ
1 - ADP
2 - ADV
3 - AUX
4 - CCONJ
5 - DET
6 - INTJ
7 - NOUN
8 - NUM
9 - PART
10 - PRON
11 - PROPN
12 - PUNCT
13 - SCONJ
14 - SYM
15 - VERB
16 - X
Qual é a categoria de 'Folha'? (Q para sair) 11


AttributeError: 'set' object has no attribute 'insert'

In [286]:
for sent in predicted:
    if any(predicted_tag == "_" for _,predicted_tag in sent):
      print(str(sent))
      break

[('Folha', '_'), ('--', 'PUNCT'), ('Como', '_'), ('você', '_'), ('recebeu', '_'), ('a', 'DET'), ('notícia', '_'), ('de', 'ADP'), ('que', '_'), ('seria', '_'), ('substituído', '_'), ('?', 'PUNCT')]


In [ ]:
test_sents[0]
sent = ""
for word in test_sents[0]:
  sent += word + " "
print(sent)

Folha -- Como você recebeu a notícia de que seria substituído ? 


## Fazendo nosso lematizador e analisador de flexões

O lematizador e a análise flexional podem ser realizados em conuunto, uma vez que determinar as flexões nos permitem "desfazê-las", i.e. obter uma versão "normalizada" do item lexical